In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from vastai.api import VastClient
# import time

In [160]:
def monitor_output(instance, command_string):
    """ Runs command on a remote machine and monitors output.
    """
    sess=instance.pb_remote.session()
    process=sess.popen(command_string)
    stderr_closed=False
    stdout_closed=False
    while True:
        try:
            err=process.stderr.readline()
            if err:
                print(err.decode(),end='')
        except EOFError:
            stderr_closed=True
        try:
            line=process.stdout.readline()
            if line: 
                print(line.decode(),end='')
            else:
                break
        except EOFError:
            stdout_closed=True
        if stderr_closed and stdout_closed:
            break
    sess.close()

In [4]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Search for Available Machines

In [15]:
def filter_offers():
    return [offer for offer in vast.search_offers('dph', 'dph<.12 inet_down>200', instance_type='on-demand') 
        if offer['gpu_name'] and offer['id'] not in []]

In [ ]:
for offer in filter_offers():
    print(offer['id'], offer['dph_total'], offer['gpu_name'], offer['inet_down'], offer['inet_down'])

## Start new Instance with Mongodb

In [11]:
offer = 375076
# offer = filter_offers()[0]['id']
onstart = """#!/bin/sh
touch ~/.no_auto_tmux
apt-get install -y mongodb-server git vim
service mongodb start
pip install --upgrade pip pipenv
"""
resp=vast.create_instance(offer, disk=20, onstart_cmd=onstart,
                          image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
mongo_instance_id = resp['new_contract']

mongo_instance = vast.get_instance(mongo_instance_id, retries=4).wait_until_running()

print(mongo_instance.pb_remote['tail']('/var/log/mongodb/mongodb.log'))

https://vast.ai/api/v0/asks/375076/?api_key=e794a3273e5a3e74d545bbc368849ff263f726042cf92587df9519a6cda62e74 
 {"client_id": "me", "image": "tensorflow/tensorflow:1.14.0-gpu-py3-jupyter", "price": null, "disk": 20, "label": null, "onstart": "#!/bin/sh\ntouch ~/.no_auto_tmux\napt-get install -y mongodb-server git vim\nservice mongodb start\npip install --upgrade pip pipenv\n", "runtype": "ssh", "python_utf8": false, "lang_utf8": false, "use_jupyter_lab": false, "jupyter_dir": null, "create_from": null, "force": false}
{"success": true, "new_contract": 400808}
Waiting 60s...
Waiting 60s...
2019-10-26T02:22:28.598+0000 I STORAGE  [initandlisten] wiredtiger_open config: create,cache_size=7452M,session_max=20000,eviction=(threads_min=4,threads_max=4),config_base=false,statistics=(fast),log=(enabled=true,archive=true,path=journal,compressor=snappy),file_manager=(close_idle_time=100000),statistics_log=(wait=0),verbose=(recovery_progress),
2019-10-26T02:22:29.163+0000 I CONTROL  [initandlisten

In [12]:
mongo_instance.ssh_connection_command

'ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai'

In [ ]:
# onstart = "#!/bin/sh\n"\
#           "touch ~/.no_auto_tmux\n"\
#           "apt-get install -y mongodb-server\n"\
#           "mkdir -p ~/.ssh\n"\
#           "ssh-keygen  -P '' -f ~/.ssh/mongo\n"
# resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
#                           image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
# worker_instance_id = resp['new_contract']

# time.sleep(30)
# worker_instance = vast.get_instance(worker_instance_id)

## Install model and download data

In [161]:
install_script = """cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pipenv install
pipenv run python setup.py develop
pipenv run download_data
"""
# monitor_output(mongo_instance, install_script)
result=mongo_instance.pb_remote.session().run(install_script)

Using TensorFlow backend.
Installing dependencies from Pipfile.lock (aadd7b)…
/root/.local/share/virtualenvs/avian_vocalizations--GzUH1gw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
To activate this project's virtualenv, run pipenv shell.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
Alternatively, run a command inside the virtualenv with pipenv run.
/root/.local/share/virtualenvs/avian_vocalizations--GzUH1gw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
running develop
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
running egg_info
/root/.local/share/virtualenvs/avian_vocalizations--GzUH1gw/lib/python3.6/site-

In [19]:
result = mongo_instance.pb_remote.session().run(
    "cd ~/avian_vocalizations;"
    "pipenv run hyperopt-mongo-worker --mongo=localhost:27017/hyperopt "
    "--max-consecutive-failures=1 --reserve-timeout=36600 --workdir=.;")

'pipenv run hyperopt-mongo-worker --mongo=localhost:27017/hyperopt --max-consecutive-failures=1 --reserve-timeout=36600;'

In [152]:
mongo_instance.ssh_connection_command


'ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai'

In [10]:
mongo_instance.wait_until_running()

400500: running   $0.0921/hr ssh5.vast.ai:10500  343.6↑ 434.2↓  flops:9.295104T
	1X 7.9GB GTX 1070 Ti, 10.666656/32X 23.8GB AMD Ryzen Threadripper 1950X 16-Core Processor
	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter

In [11]:
mongo_instance.get_ssh_connection_command()

'ssh root@ssh5.vast.ai -p 10500 -i /home/sam/.ssh/vastai'

In [40]:
# worker_instance.wait_until_running()

status is 'loading', waiting 60s...


       ID Machine   Status  Num        Model Util. % vCPUs   RAM Storage      SSH Addr SSH Port    $/hr                                         Image Net up Net down     R
0  398092     940  running  1 x  GTX 1070 Ti     0.0   4.0  32.0       1  ssh5.vast.ai    26322  0.0864  tensorflow/tensorflow:1.14.0-gpu-py3-jupyter  302.2    314.6  97.8

In [26]:
print(mongo_instance.pb_remote['service'](['mongodb','status']))

[' * Checking status of database mongodb', '   ...done.', '']

In [27]:
worker_machine.path('~/.ssh/mongo.pub').isfile()

True

In [28]:
pub_key=worker_machine['cat'](worker_machine.path('~/.ssh/mongo.pub'))
pub_key

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n'

In [ ]:
mongo_machine.session().run('echo "\n%s" >> %s'%(pub_key, mongo_machine.path('~/.ssh/authorized_keys')))

In [30]:
mongo_machine['cat'](mongo_machine.path('~/.ssh/authorized_keys'))

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n\n'

In [32]:
worker_instance.get_ssh_connection_command()

'ssh root@ssh5.vast.ai -p 38754 -i /home/sam/.ssh/vastai'

In [33]:
mongo_instance.get_ssh_connection_command()

'ssh root@ssh4.vast.ai -p 38753 -i /home/sam/.ssh/vastai'

In [81]:
"%s"%mongo_machine.path('~/.ssh/mongo.pub')

'/root/.ssh/mongo.pub'

In [6]:
mongo_instance = vast.get_instance(398753)

In [7]:
worker_instance = vast.get_instance(398754)

In [8]:
mongo_instance.start().wait_until_running(check_every_s=15)
mongo_machine = mongo_instance.pb_remote()
mongo_machine

Starting instance 398753.


<ParamikoMachine paramiko://root@ssh4.vast.ai>

In [19]:
print(mongo_machine['pwd']())

/root



In [ ]:
worker_instance.start().wait_until_running()
worker_machine = mongo_instance.pb_remote()
worker_machine

In [34]:
mongo_machine['cat']("/root/.ssh/authorized_keys")

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost'

In [35]:
mongo_machine['cat']('onstart.sh')

'#!/bin/sh\napt-get install -y mongodb-server\nservice mongodb start\n'

In [36]:
mongo_instance.get_ssh_connection_command()

'ssh root@ssh4.vast.ai -p 38753 -i /home/sam/.ssh/vastai'

In [40]:
list(mongo_machine.pgrep(""))

[ProcInfo(1, 0, 'Ss+', '/bin/bash -c while [ ! -e /.launch ]; do sleep 1; done; bash /.launch'),
 ProcInfo(6, 0, 'S+', 'bash /.launch'),
 ProcInfo(16, 0, 'Ss', '/usr/sbin/sshd'),
 ProcInfo(21, 0, 'S+', 'ssh -i /etc/forward_key -N -o TCPKeepAlive=yes -o ServerAliveInterval=10 -o ServerAliveCountMax=2 ubuntu@ssh4.vast.ai -o StrictHostKeyChecking=no -o ConnectTimeout=10 -o RequestTTY=no -o ExitOnForwardFailure=yes -R \\*:38753:localhost:22'),
 ProcInfo(44, 103, 'Sl', '/usr/bin/mongod --config /etc/mongodb.conf'),
 ProcInfo(177, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(192, 0, 'Ss', '-bash'),
 ProcInfo(198, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(200, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(215, 0, 'Ss', '-bash'),
 ProcInfo(221, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(541, 0, 'Rs', '/bin/ps -e -o pid,uid,stat,args')]

In [41]:
list(worker_machine.pgrep(''))

[ProcInfo(1, 0, 'Ss+', '/bin/bash -c while [ ! -e /.launch ]; do sleep 1; done; bash /.launch'),
 ProcInfo(6, 0, 'S+', 'bash /.launch'),
 ProcInfo(16, 0, 'Ss', '/usr/sbin/sshd'),
 ProcInfo(21, 0, 'S+', 'ssh -i /etc/forward_key -N -o TCPKeepAlive=yes -o ServerAliveInterval=10 -o ServerAliveCountMax=2 ubuntu@ssh4.vast.ai -o StrictHostKeyChecking=no -o ConnectTimeout=10 -o RequestTTY=no -o ExitOnForwardFailure=yes -R \\*:38753:localhost:22'),
 ProcInfo(44, 103, 'Sl', '/usr/bin/mongod --config /etc/mongodb.conf'),
 ProcInfo(177, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(192, 0, 'Ss', '-bash'),
 ProcInfo(198, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(200, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(215, 0, 'Ss', '-bash'),
 ProcInfo(221, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(542, 0, 'Rs', '/bin/ps -e -o pid,uid,stat,args')]

In [42]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.


In [150]:
vast.get_instances()

[398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter,
 398754: offline   $0.0252/hr ssh5.vast.ai:38754  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter]

In [86]:
inst=vast.get_instances()[0]

In [147]:
inst

398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter

In [149]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.
